# Messaging

---

## Email

---

### Background
- We'll first cover some background technology terminology
- **Spam mail**--originates from a Monty Python skit where there is way too much spam served for breakfast
- **Email Client/Message User Agent/Mail User Agent (MUA)**--program used to access a user's email
     - Desktop email clients: MS Outlook, Apple Mail, Mozilla Thunderbird, etc.  Desktop client must be installed and configured.  They must be connected to the internet to send and receive email.  However, because they store copies of email locally, a user can access emails when not connected to the internet.
    - Web browser email clients: Gmail, iCloud, Yahoo!, Outlook.com (Live Mail, Hotmail), Mail.com, Proton Mail, AOL Mail, etc.  Can access email anywhere, but must always be connected to the internet.
- **SMTP**--Simple Mail Transfer Protocol. Protocol for *sending* emails.  Dictates how message is formatted, encrypted, and relayed between mail servers.
    - Port 25 for non-encrypted emails
    - Port 465/587 for encrypted emails
- **Mail Transfer Agent (MTA)/Mail Server/SMTP Server**--middle man computers that hand off emails as they make their way across (routed across) the internet.  Uses SMTP.  Different email providers have different mail servers:
    - smtp.gmail.com
    - smtp-mail.outllook.com
    - smtp.mail.yahoo.com
    - smtp.att.net
    - smtp.comcast.net
    - smtp.verizon.net
- **IMAP**--Internet Message Access Protocol.  Protocol for *receiving* emails.  With IMAP emails are stored on a remote IMAP server and are not stored by the email client on local devices.  Because emails are stored on a remote server, its easy for multiple devices to access the emails.  
    - Port 143 for non-encrypted emails
    - Port 993 for encrypted emails
- **IMAP Servers**--different email providers use different IMAP servers.  Popular ones are:
    - imap.gmail.com
    - imap-mail.outlook.com
    - imap.mail.yahoo.com
    - imap.mail.att.net
    - imap.comcast.net
    - incoming.verizon.net
- **POP3**--Post Office Protocol.  In a nutshell, an older, simpler version of IMAP.  Mostly replaced by IMAP as POP does not work well when users access the same email account from multiple devices.
    - Port 110 for non-encrypted emails
    - Port 996 for encrypted emails
- **MIME**--Multipurpose Internet Mail Extensions.  Internet standard  that extends the format of email messages to support non-ASCII text characters and attachments of audio, images, and video.  Also allows us to combine plaintext with HTML in emails.
- **SSL** and **TLS**--see the background reading in the *HTTP Requests* section for more info.  When users use a web browser email client they often use HTTPS.  HTTPS is secure because it uses TLS for encryption.  TLS has replaced SSL, but the term SSL is still often used (even if we are talking about TLS).
- Steps to send and receive an email are as follows:
    1. Sender types up email in email client.  Email uses MIME format.  Clicks send.
    1. Email routed to multiple MTA servers using SMTP as it makes its way across the internet.
    1. Email sent to IMAP server where it is stored
    1. Recipient email client accesses email using IMAP
- **Transactional Email**--autogenerated and sent email that a user receives as part of a transaction.  It could be confirmation email for an account creation, payment receipt, password resets, etc.
- **Transactional Email Services**--paid for service that will send transactional emails for us.  They also track delivery statistics like delivery rate.  These services usually have free plans so one can test out the service.  There are many transactional service companies including:
    - Sengrid
    - Sendinblue
    - Mailgun
    - Mailjet
    - Amazon SES
- Many of these, like Sengrid, have their own Python modules

---

### Python Modules

- The following are some popular libraries for working with emails.  We'll only be sending emails so we won't cover the `imaplib` module.
- Libraries include:
    - `ssl`--built-in module for TLS
    - `smtplib`--built-in module for sending emails
    - `imaplib`--built-in module for receiving emails
    - `email`--built in module for editing and parsing emails
    - `yagmail`--third-party library designed for sending emails with Gmail.  Stands for Yet Another Gmail/SMTP client.
- Additional third-party libraries like `yagmail` are useful because many popular email providers use APIs.  Third-party libraries are written to abstract some of the API complexity away from the user.
- When using user created scripts to send and receive emails:
    1. Create a new email account.  This prevents headaches that may result from accidentally showing username and password information or accidentally sending too many emails that get flagged as spam.  It will also prevent our personal inbox from filling up with script related emails.  If we are creating a Gmail account we also need to enable "Less secure app access" for the account.
    2. Do NOT put user name and password information in source code in case others view or use code.  If needed, have users input username and password information.  If it is only stored in a variable the Python garbage collector will delete it from memory when the function or script ends.
    3. Alternatively there are different methods of account authorization like the use of private "keys" (cryptography term).

Code | Use
--- | ---
`getpass` | Module
`getpass.getpass(prompt="<PROMPT">)` | Prompt user for password.  Does not "echo" (show) password on screen when it is entered.

- **Context** is a collection of settings, ciphers, protocol versions, and trusted certificates.  A context can be reused when opening other socket connections.

Code | Use
--- | ---
`ssl` | Module
`ssl.create_default_context()` | Return SSLContext object.  This is new context with secure default settings. Optional arguments change settings.

Code | Use
--- | ---
`smtplib` | Module
`with smtplib.SMTP_SSL() as <SERVER_OBJECT>`: | Return SMTP_SSL object.  Initiates TLS-encrypted socket connection with a SMTP mail server. When we de-indent the socket connection is closed.  Argument are `"<SMTP_SERVER_URL>", <PORT_#>, context=<SSLCONTEXT_OBJECT>` 
`.login("<SENDER_ADDRESS>", "<SENDER_PASSWORD>")` | SMTP_SSL object method. Log into SMTP email server.
`.sendmail("<SENDER_ADDRESS>", "<RECIPIENT_ADDRESS>", "<MESSAGE>")` | SMTP_SSL object method.  Send message.

- The `yagmail` module can be used to easily send emails from Gmail
- **Magical Contents**--`yagmail` smartly guesses the format of the `contents=` string argument.
    1. If it can be treated as a file, like `<PATH>/<FILENAME>`, then it will be
    1. If not a file, if it can be treated as HTML then it will be
    1. If neither, it will be treated as plaintext
- Note that `yagmail` recommends one uses a private keys and keyring instead of a password.  For simplicity sake in the example we will enter our password.  For security, it will be inputted and will not be typed in the script.

Code | Use
--- | ---
`yagmail` | Module
`yagmail.SMTP('<SENDER_ADDRESS>', '<SENDER_PASSWORD>')` | Return SMTP object
`yagmail.register('<SENDER_ADDRESS>', '<SENDER_PASSWORD>')` | Wrapper for `keyring.set_password('yagmail', '<SENDER_ADDRESS>', '<SENDER_PASSWORD>')`.  Access OS keyring with Python script and store account info there.  Generate key that will be used to login to email account instead of using password.  I assume this generates a public-private key pair, stores the private key in the keyring and sends it to Gmail when the SMTP object
`.send()` | Yagmail object method.  Arguments are `to=`, `subject=`, `contents=`, `attachments=`, `cc=`, `bcc=`, `preview_only=`, and `headers=`.   Not sure on the use of the last two.  After the equal sign we would put a string value.  We could put a list of string values if there are more than one (e.g. multiple recipients).

---

**EXAMPLES**

**Note that examples will not run correctly when run on Binder**

**Send Plaintext Email**

In [ ]:
import ssl
import smtplib
import pyinputplus as pyip
import getpass

s_server_prompt = (
        'Input mail server and press Enter.'
        '  E.g. if you are using a Gmail account, type smtp.gmail.com.  '
) 
s_server = pyip.inputStr(prompt= s_server_prompt)
s_port_prompt = (
        'Input mail server port number and press Enter.'
        ' E.g. if you are using a Gmail account, type 465.  '
)
i_port = pyip.inputInt(prompt= s_port_prompt)
s_sender = pyip.inputStr(prompt= "Input your email address and press Enter:  ")
s_password = getpass.getpass(prompt= "Input your email password and press Enter:  ")
s_recipient = pyip.inputStr(prompt= "Input the recipient's email address and press Enter:  ")
s_message = (
        'Subject: Testing Testing 123\n\n'
        'This is the body of a test email message.'
)

context_object = ssl.create_default_context()
print(type(context_object))

with smtplib.SMTP_SSL(s_server, i_port, context=context_object) as smtp_server_object:
    print(type(smtp_server_object))
    smtp_server_object.login(s_sender, s_password)
    smtp_server_object.sendmail(s_sender, s_recipient, s_message)

**Send Plaintext Yagmail Email**

In [ ]:
import yagmail
import pyinputplus as pyip
import getpass

s_sender = pyip.inputStr(prompt= "Input your email address and press Enter:  ")
s_password = getpass.getpass(prompt= "Input your email password and press Enter:  ")
s_recipient = pyip.inputStr(prompt= "Input the recipient's email address and press Enter:  ")
s_subject = "Testing Testing 123'"
s_contents = 'This is the body of a test email message created with yagmail.'

yag = yagmail.SMTP(s_sender, s_password)
print(type(yag))
yag.send(to = s_recipient, subject = s_subject, contents = s_contents)

**Send Yagmail Email with Attachment**

In [ ]:
import yagmail
import pyinputplus as pyip
import getpass

s_sender = pyip.inputStr(prompt= "Input your email address and press Enter:  ")
s_password = getpass.getpass(prompt= "Input your email password and press Enter:  ")
s_recipient = pyip.inputStr(prompt= "Input the recipient's email address and press Enter:  ")
s_subject = "Testing Testing 123'"
s_contents = """\
<!DOCTYPE html>
<html lang="en-US">
    <head> </head>
    <body>
        <p>My first paragraph.</p>
        <p> My second paragraph.</p>
        <a href="https://wikipedia.org">This is a link</a>
        <p> Paragraph with <br> line break in the middle of it</p>
    </body>
</html>
"""
s_attachments = "input/monty_python.jpg"

yag = yagmail.SMTP(s_sender, s_password)
yag.send(to = s_recipient, subject = s_subject, contents = s_contents, attachments = s_attachments)

---

## Text

---

### Backgrouond
- **SMS**--Short Message Service.  Text messaging service component of most cell service systems.  Allows users to send and receive messages up to 160 alphanumeric characters to and from Global System for Mobile Communications (GSM) mobile devices.  SMS message is not guaranteed and about 1% of messages are lost entirely.
- **MMS**--Multimedia Messaging Service.  Extends the SMS to allow for longer texts and multiple forms of media like pictures and videos.  
- **Internet Messaging**--the main alternative to SMS and MMS is internet protocol based messaging services like Apple's iMessage, Facebook Messenger, or WhatsApp
- **SMS (and/or MMS) Email Gateway**--email server that a cell phone provider sets up to receive a message via email that it then forwards to the recipient as a text.  Some gateways work with both SMS and MMS while others work for either SMS or MMS.
- *Text to email.*  Write text as normal. For recipient phone number use recipient email address.
- *Email to text.*  Write an email as normal.  For the recipient's email address use the recipient's 10 digit phone number followed by @, followed by a mobile provider specific domain.  E.g.
    - Verizon SMS: ##########@vtext.com
    - Verizon MMS: ##########@vzwpix.com
    - AT&T SMS: ##########@txt.att.net
    - AT&T MMS: ##########@mms.att.net
- Many websites allow us to find a phone number's mobile provider.  These websites may also provide an API so that programs (like a Python script) can identify mobile providers.
- **SMS (and/or MMS) Gateway Service**--company that will send text messages as a service.  The client has a program (like a Python script) that communicates with the gateway service through an API.  The gateway service then sends the text through mobile providers.  One advantage is that the gateway service identifies the mobile carrier of each phone number.  It is also possible to receive texts through gateway providers, but that is more complicated.  There are tons of gateway service companies including:
    - Twilio
    - ClickSend
    - Vonage

---

### Python Modules

- We will not show any examples.  However, note that it is easy to use the ##########@domain format for the recipient email and send an SMS text.  I am unsure on compatibility of MIME emails with MMS.